<a href="https://colab.research.google.com/github/JanePer/AI---PORTFOLIO/blob/main/Task4(Pro)_%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pro**

#Классификация текстов писателей

**1. Импорт библиотек.**

In [ ]:
from google.colab import files
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
%matplotlib inline

from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Embedding, SpatialDropout1D, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**2. Загрузка базы текстов.**

In [ ]:
!rm -R '/content/texts' #remove texts directory
!unzip -q '/content/drive/MyDrive/Colab Notebooks/Тексты писателей.zip' -d '/content/texts' #unzip to created `texts` directory

In [ ]:
def readtext(file_name): #read all text in 1 raw
  file=open(file_name, 'r')
  text=file.read()
  text=text.replace("\n"," ")
  return text

authors = ["О. Генри", "Стругацкие", "Булгаков", "Саймак", "Фрай", "Брэдберри"] #list of unique authors
authors_qty=len(authors)

In [ ]:
train_text=[]
test_text=[]

for i in authors:
  for t in os.listdir('/content/texts/Тексты писателей'):
    if i in t:
      if "Обучающая" in t:
        train_text.append(readtext('/content/texts/Тексты писателей/'+t))
      else:
        test_text.append(readtext('/content/texts/Тексты писателей/'+t))
print(len(train_text))
print(len(train_text[0]))

6
1049517


**3. Нормирование и преобразование данных.**

In [ ]:
max_words_qty=20000
toks=Tokenizer(num_words=max_words_qty, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=" ",
                 oov_token='unknown', char_level=False) #create tokenizer model based on words frequency
toks.fit_on_texts(train_text) #apply tokens to train texts
freq_items=list(toks.word_index.items()) #extracting words with their indexes
print(len(freq_items))
freq_items[:10]

133070


[('unknown', 1),
 ('и', 2),
 ('в', 3),
 ('не', 4),
 ('я', 5),
 ('что', 6),
 ('на', 7),
 ('с', 8),
 ('он', 9),
 ('а', 10)]

In [ ]:
train_word_indexes=toks.texts_to_sequences(train_text) #convert train texts into indexes
test_word_indexes=toks.texts_to_sequences(test_text) #convert test texts into indexes
print(train_text[2][:100])
print(train_word_indexes[2][:20])

﻿Белая гвардия   Посвящается[1]  Любови Евгеньевне Белозерской[2]  Пошел мелкий снег и вдруг повалил
[2939, 5529, 1, 1508, 1, 1, 16735, 1515, 353, 8296, 1267, 2, 121, 15326, 1, 1, 675, 12013, 10580, 17342]


**4. Статистика по данным.**

In [ ]:
symbols_train_text=0
words_train_text=0
print("Train texts statistic:")
for i in range(authors_qty):
  print(authors[i]," "*(10-len(authors[i])), len(train_text[i]), "symbols, ", len(train_word_indexes[i]), "words")
  symbols_train_text+=len(train_text[i]) #sum overall qty of symbols within all authors train texts
  words_train_text+=len(train_word_indexes[i]) #sum overall qty of words within all authors test texts                  

print('-------')
print("Overall symbols: ", symbols_train_text, " Overall words: ", words_train_text,) 
print('\n')

symbols_test_text=0
words_test_text=0
print("Test texts statistic:")
for i in range(authors_qty):
  print(authors[i]," "*(10-len(authors[i])), len(test_text[i]), "symbols, ", len(test_word_indexes[i]), "words")
  symbols_test_text+=len(test_text[i]) #sum overall qty of symbols within all authors train texts
  words_test_text+=len(test_word_indexes[i]) #sum overall qty of words within all authors test texts                  

print('-------')
print("Overall symbols: ", symbols_test_text, " Overall words: ", words_test_text,) 

Train texts statistic:
О. Генри    1049517 symbols,  160607 words
Стругацкие  2042469 symbols,  313012 words
Булгаков    1765648 symbols,  261465 words
Саймак      1609507 symbols,  251502 words
Фрай        3700010 symbols,  568533 words
Брэдберри   1386454 symbols,  214454 words
-------
Overall symbols:  11553605  Overall words:  1769573


Test texts statistic:
О. Генри    349662 symbols,  53238 words
Стругацкие  704846 symbols,  108621 words
Булгаков    875042 symbols,  132730 words
Саймак      318811 symbols,  50360 words
Фрай        1278191 symbols,  196731 words
Брэдберри   868673 symbols,  132524 words
-------
Overall symbols:  4395225  Overall words:  674204


**5. Создание обучающей и проверочной выборки.**

In [ ]:
def getSetIndexes(wordIndexes, xLen, step): #cut separate author's text into xLen samples
  x_sample=[]
  wordslen=len(wordIndexes)
  index=0
  while (index+xLen)<=wordslen:
    x_sample.append(wordIndexes[index:index+xLen])
    index+=step
  return x_sample

def getAllSetsIndexes(wordIndexes, xLen, step): #create X set cut into defined length of samples and Y set with correspond author within all authors
  classes=len(wordIndexes) #6 authors
  class_x_sample=[]
  for w in wordIndexes: #take each list of 6 lists
    class_x_sample.append(getSetIndexes(w, xLen, step)) #cut each text list into samples and append to summary list of lists

  X_sam=[]
  Y_sam=[]
  for c in range(classes): #within 6 authors
    cx=class_x_sample[c] #all cut samples of each author's list
    for x in range(len(cx)): #within qty of all samples of each author
      X_sam.append(cx[x]) #append all cut samples to final list
      Y_sam.append(utils.to_categorical(c, classes)) #append all correspond author's vectors to final list

  X_sam=np.array(X_sam)
  Y_sam=np.array(Y_sam)
  return (X_sam, Y_sam)

In [ ]:
xLen=6300
step=70
x_train1, y_train1=getAllSetsIndexes(train_word_indexes, xLen, step)
x_test, y_test=getAllSetsIndexes(test_word_indexes, xLen, step)
print(x_train1.shape)
print(y_train1.shape)
print(x_test.shape)
print(y_test.shape)

(24742, 6300)
(24742, 6)
(9095, 6300)
(9095, 6)


In [ ]:
x_train, x_val, y_train, y_val=train_test_split(x_train1, y_train1, test_size=0.2, shuffle=True)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(19793, 6300)
(19793, 6)
(4949, 6300)
(4949, 6)


In [ ]:
x_train01=toks.sequences_to_matrix(x_train.tolist()) #convert indexes into bag of words vectors
x_val01=toks.sequences_to_matrix(x_val.tolist())
x_test01=toks.sequences_to_matrix(x_test.tolist())
print(x_train01.shape)
print(x_train01[0][:50])

(19793, 20000)
[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]


**6. Создание и обучение сети.**

In [ ]:
model=Sequential()
model.add(Dense(400, input_dim=max_words_qty, activation='sigmoid'))
model.add(Dense(200, activation='sigmoid'))
model.add(Dense(200, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 400)               8000400   
_________________________________________________________________
dense_1 (Dense)              (None, 200)               80200     
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 1206      
Total params: 8,122,006
Trainable params: 8,122,006
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=model.fit(x_train01, y_train, epochs=10, batch_size=128, validation_data=(x_val01, y_val), verbose=1)

Epoch 1/10
155/155 [==============================] - 5s 15ms/step - loss: 0.2211 - accuracy: 0.9483 - val_loss: 0.0063 - val_accuracy: 1.0000
Epoch 2/10
155/155 [==============================] - 2s 11ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 3/10
155/155 [==============================] - 2s 11ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 4/10
155/155 [==============================] - 2s 11ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 7.4722e-04 - val_accuracy: 1.0000
Epoch 5/10
155/155 [==============================] - 2s 11ms/step - loss: 8.6734e-04 - accuracy: 1.0000 - val_loss: 5.1617e-04 - val_accuracy: 1.0000
Epoch 6/10
155/155 [==============================] - 2s 11ms/step - loss: 6.3295e-04 - accuracy: 1.0000 - val_loss: 3.7821e-04 - val_accuracy: 1.0000
Epoch 7/10
155/155 [==============================] - 2s 11ms/step - loss: 4.7864e-04 - accuracy: 1.0000 - val_loss: 2.892

**7. Распознавание проверочной выборки**

In [ ]:
model.evaluate(x_test01,y_test, verbose=1)

285/285 [==============================] - 1s 3ms/step - loss: 0.0743 - accuracy: 0.9763


[0.07427144050598145, 0.9762507081031799]

Up to 98% correct answers for testing sample, not enough RAM to display accuracy by authors separately.